# How to reproduce the creation of CryptoGDelt2022.csv

* __1st run CryptoGDelt2022_GDELT_initial_download.ipynb:__ this code downloads the whole period of news from Gdelt - it may take more than a week to do so, recommended to skip this part for reproduction
(development dataset: None; deployment dataset - input: None - output: OneYearNewsDataset.csv)

* __2nd run Relevance/ToReproduceRelevance.ipynb:__ this code train the supervised relevance score using all_score_training.csv dataset and deploy it to the pipe of news downloaded from Gdelt in an addictive manner
(development dataset: Relevance/all_score_training.csv; deployment dataset - input: OneYearNewsDataset.csv - output: OneYearNewsDataset_AfterRelevance.csv)

* __3rd run Sentiment/ToReproduceSentiment.ipynb:__ this code retrain Finbert sentiment score using CryptoLin_IE_v2.csv dataset and deploy it to the pipe of news downloaded from Gdelt in an addictive manner - it may take more than 12 hours to run this step
(development dataset: Sentiment/CryptoLin_IE_v2.csv; deployment dataset - input: OneYearNewsDataset_AfterRelevance.csv - output: OneYearNewsDataset_AfterSentiment.csv)

* __4th run Sentiment/ToReproduceStrength.ipynb:__ this code train the Strength score based on Fama French using CryptoLin_IE_v2.csv dataset and deploy it to the pipe of news downloaded from Gdelt in an addictive manner
(development dataset: Strength/FF3_daily.csv and Strength/news_short.xlsx; deployment dataset - input: OneYearNewsDataset_AfterSentiment.csv - output: OneYearNewsDataset_AfterStrength.csv)

__OneYearNewsDataset_AfterStrength.csv is ready:__ OneYearNewsDataset_AfterStrength.csv is similar to CryptoGDelt2022.csv, CryptoGDelt2022.csv contains some extra date fields used for sorting the news by date and time.


* __5th run CryptoGDelt2022_EDA.ipynb:__ this code presents the EDA of the generated dataset.



In [1]:
!pip install gdeltdoc

In [2]:
from datetime import date, timedelta, datetime
start_date = datetime.strptime('01/12/2022 00:00:00', '%d/%m/%Y %H:%M:%S')
end_date = datetime.strptime('31/12/2023 00:00:00', '%d/%m/%Y %H:%M:%S')

In [3]:
print(str(start_date.strftime("%Y%m%d%H%M%S")))

20221201000000


In [4]:
num_of_days = (end_date - start_date).days+1
num_of_days

396

In [9]:
# bitcoin%20OR%20cryptocurrency%20OR%20cryptocurrencies

import pandas as pd

from gdeltdoc import GdeltDoc, Filters

currentday = end_date
previousday = end_date - timedelta(days=1)

f = Filters(
    keyword = "sanofi",
    start_date = str(previousday.strftime("%Y%m%d")),
    end_date = str(currentday.strftime("%Y%m%d"))
)

gd = GdeltDoc()

# Search for articles matching the filters
articles = gd.article_search(f)
df = articles[(articles["language"] == 'English')]
df

,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry
0,https://www.gdnonline.com/Details/1299368,,International Business : Drugmakers set to rai...,20231230T140000Z,https://www.gdnonline.com/gdnimages/20231230%5...,gdnonline.com,English,United States
1,https://www.gdnonline.com/Details/1299368/Drug...,,International Business : Drugmakers set to rai...,20231230T111500Z,https://www.gdnonline.com/gdnimages/20231230%5...,gdnonline.com,English,United States
4,https://finance.yahoo.com/news/3-stocks-could-...,,3 Stocks That Could Be Monster Winners in 2024,20231230T121500Z,https://s.yimg.com/ny/api/res/1.2/sJdvz6OLubCK...,finance.yahoo.com,English,United States


In [11]:
import time
for i in range(num_of_days):
    currentday = end_date- timedelta(days=i)
    previousday = end_date- timedelta(days=i+1)
    print("start_date, end_date = ",previousday,currentday)
    for item in ['sanofi']:
        print(item)
        try:
            f = Filters(
                keyword = item,
                start_date = str(previousday.strftime("%Y%m%d")),
                end_date = str(currentday.strftime("%Y%m%d"))
            )

            gd = GdeltDoc()
            # Search for articles matching the filters
            articles = gd.article_search(f)
            articles = articles[(articles["language"] == 'English')]
            df = pd.concat([df, articles], axis=0)
            df = df.drop_duplicates(subset=['url'], keep='last')
            time.sleep(1)
        except:
            pass
    time.sleep(5) # Sleeping 5 seconds to avoid being blocked by Gdelt/Gdeltdoc


start_date, end_date =  2023-12-30 00:00:00 2023-12-31 00:00:00
sanofi
start_date, end_date =  2023-12-29 00:00:00 2023-12-30 00:00:00
sanofi
start_date, end_date =  2023-12-28 00:00:00 2023-12-29 00:00:00
sanofi
start_date, end_date =  2023-12-27 00:00:00 2023-12-28 00:00:00
sanofi
start_date, end_date =  2023-12-26 00:00:00 2023-12-27 00:00:00
sanofi
start_date, end_date =  2023-12-25 00:00:00 2023-12-26 00:00:00
sanofi
start_date, end_date =  2023-12-24 00:00:00 2023-12-25 00:00:00
sanofi
start_date, end_date =  2023-12-23 00:00:00 2023-12-24 00:00:00
sanofi
start_date, end_date =  2023-12-22 00:00:00 2023-12-23 00:00:00
sanofi
start_date, end_date =  2023-12-21 00:00:00 2023-12-22 00:00:00
sanofi
start_date, end_date =  2023-12-20 00:00:00 2023-12-21 00:00:00
sanofi
start_date, end_date =  2023-12-19 00:00:00 2023-12-20 00:00:00
sanofi
start_date, end_date =  2023-12-18 00:00:00 2023-12-19 00:00:00
sanofi
start_date, end_date =  2023-12-17 00:00:00 2023-12-18 00:00:00
sanofi
start_

In [12]:
df = df.reset_index(drop=True)

In [13]:
df.to_csv("sanofi2023.csv", index=False)

In [14]:
df.tail()

,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry
10912,https://www.newyorktelegraph.com/news/27313732...,,Pharmacovigilance Market is Forecasted to Atta...,20221130T234500Z,,newyorktelegraph.com,English,United States
10913,https://www.texasguardian.com/news/273137325/p...,,Pharmacovigilance Market is Forecasted to Atta...,20221201T000000Z,,texasguardian.com,English,United States
10914,https://www.tmcnet.com/usubmit/2022/11/30/9721...,,Tencent Highlights Its Global Vision and Strat...,20221130T070000Z,https://mma.prnewswire.com/media/1958530/Tence...,tmcnet.com,English,United States
10915,https://www.tmcnet.com/usubmit/2022/11/30/9721...,,BioNTech and Ryvu Therapeutics Enter into Glob...,20221130T070000Z,https://images.tmcnet.com/tmc/world.jpg,tmcnet.com,English,United States
10916,https://www.elpasoinc.com/biontech-and-ryvu-th...,,BioNTech and Ryvu Therapeutics Enter into Glob...,20221130T073000Z,https://bloximages.newyork1.vip.townnews.com/e...,elpasoinc.com,English,United States
